In [1]:
# Data Preparation, Training and evaluation
import numpy as np
import cv2

image = cv2.imread('images/digits.png')
gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
small = cv2.pyrDown(image)
cv2.imshow('Digits Image', small)
cv2.waitKey(0)
cv2.destroyAllWindows()